In [3]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timezone

In [6]:
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_seq_items', 500)

# High Dividend Yield Stocks


## Get Fundamental

In [25]:
from alphalib.analysis import get_yfinance

In [26]:
stock_info = get_yfinance("BAK")

In [27]:
print(stock_info)

YahooFinance(symbol='BAK', name='Braskem SA', exchange='NYQ', sector='Basic Materials', currentPrice=13.44, fiveYearAvgDividendYield=None, earningsDate=datetime.datetime(1, 1, 1, 0, 0), exDividendDate=datetime.datetime(2022, 4, 20, 8, 0), dividendDate=datetime.datetime(2022, 5, 12, 8, 0), lastDividendDate=datetime.datetime(2021, 12, 10, 8, 0), forwardEps=1.1, forwardPE=12.218182, trailingEps=3.76, trailingPE=3.574468, pegRatio=0.57, priceToBook=0.56508577, freeCashflow=12197989376, returnOnEquity=0.85393, debtToEquity=659.908, priceToSalesTrailing12Months=0.044267736, payoutRatio=0.8911, dividendYield=0.2062, dividendRate=2.63)


In [28]:
stock_info.to_df().T.head(100)

,0
symbol,BAK
name,Braskem SA
exchange,NYQ
sector,Basic Materials
currentPrice,13.44
fiveYearAvgDividendYield,None
earningsDate,0001-01-01 00:00:00
exDividendDate,2022-04-20 08:00:00
dividendDate,2022-05-12 08:00:00
lastDividendDate,2021-12-10 08:00:00


## Dividend History and Yield

In [29]:
from alphalib.analysis import get_nasdaq

In [30]:
stock_info = get_nasdaq("GOGL")

In [31]:
stock_info.to_df().T.head(10)

,0
label,GOGL Dividend History
symbol,GOGL
exDividendDate,2022-09-06 00:00:00
dividend_yield_pct,27.84
annual_dividend,2.4
pe_ratio,2.56
url,https://www.nasdaq.com/market-activity/stocks/...


In [32]:
stock_info.dividend_history.head(100)

,ex_eff_date,type,cash_amount,declaration_date,record_date,payment_date
0,2022-09-06,CASH,0.600,2022-08-25 00:00:00,2022-09-07,2022-09-14 00:00:00
1,2022-05-31,CASH,0.500,2022-05-19 00:00:00,2022-06-01,2022-06-08 00:00:00
2,2022-03-02,CASH,0.900,2022-02-16 00:00:00,2022-03-03,2022-03-10 00:00:00
3,2021-12-08,CASH,0.850,2021-11-24 00:00:00,2021-12-09,2021-12-16 00:00:00
4,2021-09-09,CASH,0.500,2021-08-26 00:00:00,2021-09-10,2021-09-20 00:00:00
5,2021-06-02,CASH,0.250,2021-05-20 00:00:00,2021-06-03,2021-06-10 00:00:00
6,2020-03-05,CASH,0.050,2020-02-18 00:00:00,2020-03-06,2020-03-19 00:00:00
7,2019-12-02,CASH,0.150,2019-11-21 00:00:00,2019-12-03,2019-12-18 00:00:00
8,2019-08-29,CASH,0.100,2019-08-15 00:00:00,2019-08-30,2019-09-12 00:00:00
9,2019-06-05,CASH,0.025,2019-05-22 00:00:00,2019-06-06,2019-06-20 00:00:00


# Analyse High Dividend Stocks

In [7]:
def from_epoch_time(value) -> datetime:
    return pd.to_datetime(value, unit="s")

In [8]:
COUNTRY = "united states"

STOCK_FILE = "data/stock.xlsx"
SHEET_NAME_STOCK = "stock"

STOCK_INFO_FILE = "data/stock_info.xlsx"
SHEET_NAME_STOCK_INFO = "stock_info"

STOCK_STATS_FILE = "data/stock_stats.xlsx"
SHEET_NAME_STOCK_STATS = "stock_stats"

STOCK_DIVIDEND_FILE = "data/stock_dividends.xlsx"
SHEET_NAME_STOCK_DIVIDEND = "stock_dividends"

In [9]:
df_stock = pd.read_excel(STOCK_FILE, sheet_name=SHEET_NAME_STOCK, engine="openpyxl")
df_stock_info = pd.read_excel(STOCK_INFO_FILE, sheet_name=SHEET_NAME_STOCK_INFO, engine="openpyxl")
df_stock_stats = pd.read_excel(STOCK_STATS_FILE, sheet_name=SHEET_NAME_STOCK_STATS, engine="openpyxl")
df_stock_dividend = pd.read_excel(STOCK_DIVIDEND_FILE, sheet_name=SHEET_NAME_STOCK_DIVIDEND, engine="openpyxl")

In [10]:
# Convert field names to lower case
df_stock = df_stock.rename(columns=str.lower)
df_stock_info = df_stock_info.rename(columns=str.lower)
df_stock_stats = df_stock_stats.rename(columns=str.lower)
df_stock_dividend = df_stock_dividend.rename(columns=str.lower)

In [11]:
df_stock_stats["lastdividenddate"] = df_stock_stats["lastdividenddate"].apply(from_epoch_time)

In [12]:
df_yield = df_stock_stats.sort_values(by=["fiveyearavgdividendyield"], ascending=False)[["symbol","name","sector","fiveyearavgdividendyield", "lastdividendvalue", "lastdividenddate", "threeyearaveragereturn", "fiveyearaveragereturn" ]]

In [13]:
df_high_yield = df_yield[df_yield["lastdividenddate"].dt.year == 2022]
df_high_yield.head(15)

,symbol,name,sector,fiveyearavgdividendyield,lastdividendvalue,lastdividenddate,threeyearaveragereturn,fiveyearaveragereturn
1439,CEN,Center Coast Brookfield MLP & E,Financial Services,24.95,0.225,2022-09-13,NaN,NaN
1530,CHKR,CHESAPEAKE GRANITE WASH TR,Energy,22.62,0.085,2022-08-18,NaN,NaN
5335,NTG,"Tortoise Midstream Energy Fund,",Financial Services,20.49,0.770,2022-08-23,NaN,NaN
2893,FMO,Fiduciary/Claymore Energy Infra,Financial Services,20.12,0.245,2022-02-14,NaN,NaN
1617,CLM,111422,Financial Services,19.93,0.181,2022-09-14,NaN,NaN
1828,CRF,33138,Financial Services,19.61,0.173,2022-09-14,NaN,NaN
5632,OXLC,Oxford Lane Capital Corp.,Financial Services,18.62,0.075,2022-09-15,NaN,NaN
4092,IVR,INVESCO MORTGAGE CAPITAL INC,Real Estate,18.20,0.900,2022-07-08,NaN,NaN
5575,ORC,"Orchid Island Capital, Inc.",Real Estate,18.03,0.160,2022-08-30,NaN,NaN
2340,ECTM,ECA Marcellus Trust I,Energy,17.98,0.176,2022-08-18,NaN,NaN


In [14]:
symbols = df_high_yield["symbol"].head(15).tolist()

# By Sector

In [15]:
df_yield_sector = df_stock_stats.sort_values(by=["sector", "fiveyearavgdividendyield"], ascending=False)[["symbol","name", "sector", "fiveyearavgdividendyield", "lastdividendvalue", "lastdividenddate" ]]
df_yield_sector = df_yield_sector[ (df_yield_sector["lastdividenddate"].dt.year == 2022) & (df_yield_sector["fiveyearavgdividendyield"].notnull())]

In [22]:
df_yield_sector[(df_yield_sector["fiveyearavgdividendyield"] >= 10)].groupby(by=["sector"]).head(1000)

,symbol,name,sector,fiveyearavgdividendyield,lastdividendvalue,lastdividenddate
6937,SPH,"Suburban Propane Partners, L.P.",Utilities,11.04,0.325000,2022-08-01
4813,MINDP,"MIND Technology, Inc. - Series",Technology,11.15,0.563000,2022-04-13
4092,IVR,INVESCO MORTGAGE CAPITAL INC,Real Estate,18.20,0.900000,2022-07-08
5575,ORC,"Orchid Island Capital, Inc.",Real Estate,18.03,0.160000,2022-08-30
8038,WMC,Western Asset Mortgage Capital,Real Estate,15.40,0.400000,2022-06-30
1210,BVERS,BEAVER COAL,Real Estate,15.16,120.000000,2022-09-21
4822,MITT,"AG Mortgage Investment Trust, I",Real Estate,14.90,0.210000,2022-06-29
5437,NYMT,"New York Mortgage Trust, Inc.",Real Estate,14.09,0.100000,2022-06-24
1535,CHMI,Cherry Hill Mortgage Investment,Real Estate,13.20,0.270000,2022-06-29
584,ARR,"ARMOUR Residential REIT, Inc.",Real Estate,12.82,0.100000,2022-09-14


# Analyze High-Dividend Yield Stock

https://finance.yahoo.com/u/yahoo-finance/watchlists/high-yield-dividend-stocks/

In [ ]:
symbols = ["PBR","PBR-A", "SNPTY", "BNPQY", "NRDBY", "AMKBY", "ISNPY", "NLY", "BAK", "AGNC", "ELP", "ZIM", "SBLK", "CCU", "CLM", "GOGL", "CIM", "ARI", "MFA", "CRF", "BRMK", "EGLE", "OPI", "ARR", "LPG", "ECC", "IVR", "ORC", "BPT", "GLO"]

In [57]:
from datetime import datetime, timedelta

In [58]:
current_month = datetime.now()
next_month = current_month + timedelta(days=31)

In [66]:
for symbol in symbols:
    stock_info = get_yfinance(symbol)
    if (stock_info.earningsDate.year == current_month.year and stock_info.earningsDate.month == current_month.month) or \
       (stock_info.earningsDate.year == next_month.year and stock_info.earningsDate.month == next_month.month):
        print(f"\n\n{stock_info.name} - {stock_info.symbol}")
        print("=====================================")
        print(stock_info.to_df().T.head(100))
        
        div_info = get_nasdaq(symbol)
        print("\nDividend Details")
        print("----------------")
        print(div_info.to_df().T.head(100))
        print(div_info.dividend_history.head(10))



Annaly Capital Management Inc. - NLY
                                                           0
symbol                                                   NLY
name                          Annaly Capital Management Inc.
exchange                                                 NYQ
sector                                           Real Estate
currentPrice                                           16.64
fiveYearAvgDividendYield                               11.63
earningsDate                             2022-10-27 04:00:00
exDividendDate                           2022-09-29 08:00:00
dividendDate                             2022-10-31 08:00:00
lastDividendDate                         2022-09-29 08:00:00
forwardEps                                              3.84
forwardPE                                           4.333334
trailingEps                                               10
trailingPE                                             1.664
pegRatio                                      

Stocks to track
- PBR-A
- PBR
- GOGL
- ORC - monthly
- OXLC - monthly